# 1 Loading Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import dask
import dask.dataframe as df

# 2 Reading a part of the CSV

In [2]:
skiprows=0
nrows=1136376

In [3]:
test=pd.read_csv("../input/test-eda-part1/test.csv.gzip",compression="gzip",skiprows=skiprows,nrows=nrows)

# 3 Understanding the Data

# 3.1 Shape of the Data

In [4]:
test.shape

(1136376, 170)

# 3.2 Find the Data Types

In [5]:
test.dtypes

customer_ID     object
S_2             object
P_2            float64
D_39             int64
B_1            float64
                ...   
D_140            int64
D_141          float64
D_143            int64
D_144          float64
D_145            int64
Length: 170, dtype: object

# 3.3 Glimps of Data

In [6]:
test.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,D_41,B_3,D_43,...,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-02-19,0.631315,0,0.010728,0.814497,0.0,0.0,0.002347,1.031165,...,-1,-1,-1,-1,-1,0,0.0,-1,0.008281,-1
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-03-25,0.587042,0,0.011026,0.810848,0.0,0.0,0.009132,1.031165,...,-1,-1,-1,-1,0,0,0.0,0,0.003753,0
2,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-04-25,0.609056,0,0.016390,1.004620,0.0,0.0,0.004192,1.031165,...,-1,-1,-1,-1,0,0,0.0,0,0.002156,0
3,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-05-20,0.614911,0,0.021672,0.816549,0.0,0.0,0.015325,1.031165,...,-1,-1,-1,-1,0,0,0.0,0,0.005206,0
4,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-06-15,0.591673,8,0.015923,0.810456,0.0,0.0,0.011281,1.031165,...,-1,-1,-1,-1,0,0,0.0,0,0.007421,0


Thus we can reduce the size of the data to float16

# 4 Data Compression

In [7]:
for col in test.columns:
    if col not in ['customer_ID', 'target', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68','S_2']:
        test[col]=test[col]/(test[col].max()-test[col].min())
    if col not in ['customer_ID', 'target', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68','S_2']:
        test[col]=test[col].astype("float16")

# 5 Label encoding/factorizing the remaining character variables

The categorical features are the followings ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [8]:
columns=['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'] 
for col in columns: 
    print(test[col].value_counts())

 0    963349
 1    162530
 2     10190
-1       307
Name: B_30, dtype: int64
 2    413304
 3    277283
 1    210904
 5     88326
 7     62451
 4     49343
 6     34458
-1       307
Name: B_38, dtype: int64
 1    698399
 0    409915
-1     28062
Name: D_114, dtype: int64
 0    1105269
-1      28062
 1       3045
Name: D_116, dtype: int64
 0    313184
 5    241097
 4    228537
 3    124490
 6     98995
 7     80266
-1     28062
 2     21745
Name: D_117, dtype: int64
 0    882598
 1    225716
-1     28062
Name: D_120, dtype: int64
 2    865654
 1    231805
 0     30830
-1      8087
Name: D_126, dtype: int64
3    841163
0    185266
4     91101
1     11120
2      5021
5      2705
Name: D_63, dtype: int64
 0    600344
 3    295432
 2    202857
-1     37743
Name: D_64, dtype: int64
-1    1001277
 1     135099
Name: D_66, dtype: int64
 6    565011
 5    258919
 4    104134
 3     99040
 2     48057
-1     34650
 1     26565
Name: D_68, dtype: int64


* D_64 has category 1 missing which is present in training data set.
* D_66 has category 0 missing which is present in training dataset.
* D_68 has category 0 missing which is present in training dataset.

In [9]:
test.groupby(by="customer_ID")

Here we factorize the date column.

In [10]:
test["S_2"]=test["S_2"].str.slice(start=0,stop=7)
mapping={"2018-04":13,"2018-05":14,"2018-06":15,"2018-07":16,"2018-08":17,"2018-09":18,"2018-10":19,"2018-11":20,"2018-12":21,"2019-01":22,"2019-02":23,"2019-03":24,"2019-04":25,"2019-05":26,"2019-06":27,"2019-07":28,"2019-08":29,"2019-09":30,"2019-10":31} 
test=test.replace({"S_2":mapping}) 
test["S_2"]

0          23
1          24
2          25
3          26
4          27
           ..
1136371    26
1136372    27
1136373    28
1136374    30
1136375    31
Name: S_2, Length: 1136376, dtype: int64

In [11]:
mapping={0:0.183402,1:0.627797,2:0.612930,-1:0.356151}
test=test.replace({"B_30":mapping}) 
test["B_30"]

0          0.183402
1          0.183402
2          0.183402
3          0.183402
4          0.183402
             ...   
1136371    0.183402
1136372    0.183402
1136373    0.627797
1136374    0.183402
1136375    0.183402
Name: B_30, Length: 1136376, dtype: float64

In [12]:
mapping={2:0.063306,3:0.328604,1:0.135264,5:0.559430,4:0.699716,7:0.498865,6:0.614916,-1:0.356151}
test=test.replace({"B_38":mapping}) 
test["B_38"]

0          0.135264
1          0.135264
2          0.063306
3          0.063306
4          0.063306
             ...   
1136371    0.328604
1136372    0.559430
1136373    0.559430
1136374    0.328604
1136375    0.328604
Name: B_38, Length: 1136376, dtype: float64

In [13]:
mapping={1:0.179497,0:0.345989,-1:0.437736}
test=test.replace({"D_114":mapping}) 
test["D_114"]

0          0.437736
1          0.437736
2          0.437736
3          0.437736
4          0.345989
             ...   
1136371    0.179497
1136372    0.179497
1136373    0.179497
1136374    0.179497
1136375    0.179497
Name: D_114, Length: 1136376, dtype: float64

In [14]:
mapping={0:0.242413,-1:0.437736,1:0.613341}
test=test.replace({"D_116":mapping}) 
test["D_116"]

0          0.437736
1          0.437736
2          0.437736
3          0.437736
4          0.242413
             ...   
1136371    0.242413
1136372    0.242413
1136373    0.242413
1136374    0.242413
1136375    0.242413
Name: D_116, Length: 1136376, dtype: float64

In [15]:
mapping={0:0.273270,4:0.269573,5:0.173700,3:0.340783,6:0.135087,7:0.134076,-1:0.437736,2:0.446632}
test=test.replace({"D_117":mapping}) 
test["D_117"]

0          0.437736
1          0.437736
2          0.437736
3          0.437736
4          0.273270
             ...   
1136371    0.273270
1136372    0.273270
1136373    0.273270
1136374    0.134076
1136375    0.134076
Name: D_117, Length: 1136376, dtype: float64

In [16]:
mapping={0:0.212713,1:0.471097,-1:0.437736}
test=test.replace({"D_120":mapping}) 
test["D_120"]

0          0.437736
1          0.437736
2          0.437736
3          0.437736
4          0.471097
             ...   
1136371    0.471097
1136372    0.471097
1136373    0.471097
1136374    0.471097
1136375    0.471097
Name: D_120, Length: 1136376, dtype: float64

In [17]:
mapping={2:0.232809,1:0.309875,0:0.209538,-1:0.468044}
test=test.replace({"D_126":mapping}) 
test["D_126"]

0          0.309875
1          0.309875
2          0.309875
3          0.309875
4          0.309875
             ...   
1136371    0.232809
1136372    0.232809
1136373    0.232809
1136374    0.232809
1136375    0.232809
Name: D_126, Length: 1136376, dtype: float64

In [18]:
mapping={3:0.262817,0:0.169865,4:0.290894,1:0.190918,2:0.237401,5:0.317444}
test=test.replace({"D_63":mapping}) 
test["D_63"]

0          0.169865
1          0.169865
2          0.169865
3          0.169865
4          0.169865
             ...   
1136371    0.262817
1136372    0.262817
1136373    0.262817
1136374    0.262817
1136375    0.262817
Name: D_63, Length: 1136376, dtype: float64

In [19]:
mapping={0:0.169255,3:0.343223,2:0.315242,-1:0.408642,1:0.220723}
test=test.replace({"D_64":mapping}) 
test["D_64"]

0          0.408642
1          0.408642
2          0.408642
3          0.408642
4          0.343223
             ...   
1136371    0.169255
1136372    0.315242
1136373    0.315242
1136374    0.169255
1136375    0.169255
Name: D_64, Length: 1136376, dtype: float64

In [20]:
mapping={-1:0.257171,1:0.183504,0:0.383747}
test=test.replace({"D_66":mapping}) 
test["D_66"]

0          0.257171
1          0.257171
2          0.257171
3          0.257171
4          0.257171
             ...   
1136371    0.257171
1136372    0.257171
1136373    0.257171
1136374    0.257171
1136375    0.257171
Name: D_66, Length: 1136376, dtype: float64

In [21]:
mapping={6:0.174952,5:0.262219,3:0.369780,4:0.343603,2:0.392902,-1:0.414493,1:0.403990,0:0.179717}
test=test.replace({"D_68":mapping}) 
test["D_68"]

0          0.414493
1          0.414493
2          0.414493
3          0.414493
4          0.174952
             ...   
1136371    0.343603
1136372    0.174952
1136373    0.174952
1136374    0.262219
1136375    0.262219
Name: D_68, Length: 1136376, dtype: float64

In [22]:
x=pd.DataFrame(np.linspace(start=12,stop=0,num=13))
y=pd.DataFrame([0.258934,0.257893,0.257176,0.256362,0.254698,0.252840,0.250988,0.248830,0.245882,0.242539,0.236346,0.238807,0.232017])

mapping={}
for i in range(13,32):
    mapping[i]=(x[0].corr(y[0])*y[0].std()/x[0].std())*(i-x[0].mean())+y[0].mean()
test=test.replace({"S_2":mapping}) 
test["S_2"]

0          0.286393
1          0.288609
2          0.290826
3          0.293042
4          0.295258
             ...   
1136371    0.293042
1136372    0.295258
1136373    0.297474
1136374    0.301907
1136375    0.304123
Name: S_2, Length: 1136376, dtype: float64